## (MNIST 합성곱 실습)

#### 1. 라이브러리

In [4]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow.keras import datasets, layers, models

#### 2. 데이터 준비
    - MNIST 손글씨 이미지 데이터

In [6]:
(train_images, train_labels),(test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000,28,28,1))
test_images = test_images.reshape((10000,28,28,1))

train_images, test_images = train_images/255.0, test_images/255.0

#### 3. 합성곱 층 만들기

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
#첫번째 인자(32) : 컨볼루션 필터의 수 입니다.
#두번째 인자(3,3) : 컨볼루션 커널의 (행, 열) 입니다.
#activation : 활성화 함수 설정합니다
#‘relu’ : rectifier 함수, 은익층에 주로 쓰입니다.
#input_shape : 샘플 수를 제외한 입력 형태를 정의 합니다. 모델에서 첫 레이어일 때만 정의하면 됩니다.
#(행, 열, 채널 수)로 정의합니다. 흑백영상인 경우에는 채널이 1이고, 컬러(RGB)영상인 경우에는 채널을 3으로 설정합니다.
model.add(layers.MaxPooling2D(2,2))
# MaxPooling ; 이 레이어는 영상의 작은 변화라던지 사소한 움직임이 특징을 추출할 때 크게 영향을 미치지 않도록 합니다.
# pool_size : 수직, 수평 축소 비율을 지정합니다. (2, 2)이면 출력 영상 크기는 입력 영상 크기의 반으로 줄어듭니다.
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


#### 4. 마지막에 Dense 층 추가하기

In [10]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

#### 5. 모델 컴파일과 훈련하기

In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_images,train_labels,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 31s 16ms/step - loss: 0.3464 - accuracy: 0.8930
Epoch 2/5
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0476 - accuracy: 0.9853
Epoch 3/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0325 - accuracy: 0.9902
Epoch 4/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0231 - accuracy: 0.9927
Epoch 5/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0195 - accuracy: 0.9942


#### 6. 모델평가

In [13]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

313/313 - 1s - loss: 0.0268 - accuracy: 0.9916
